In [151]:
import numpy as np

import sys
sys.path.append('.')

from sfepy.base.base import output, IndexedStruct, Struct
from sfepy.discrete import (FieldVariable, Material, Integral, Integrals,
                            Equation, Equations, Problem)
from sfepy.discrete.fem import Mesh, FEDomain, Field
from sfepy.discrete.fem.geometry_element import geometry_data
from sfepy.terms import Term
from sfepy.discrete.conditions import Conditions, EssentialBC
from sfepy.solvers.ls import ScipyDirect
from sfepy.solvers.nls import Newton
from sfepy.mechanics.matcoefs import stiffness_from_youngpoisson
from sfepy import data_dir


#from sfepy.examples.linear_elasticity.its2D_2 import stress_strain
#from sfepy.solvers.auto_fallback import AutoDirect
from sfepy.examples.linear_elasticity.its2D_3 import nodal_stress


In [152]:
mesh = Mesh.from_file('cm_triangle2d.mesh')
domain = FEDomain('domain', mesh)

omega = domain.create_region('Omega', 'all')
top = domain.create_region('Top', 'vertex 2', 'vertex')
bottom_left = domain.create_region('Bottom_Left', 'vertex 0', 'vertex')                                        
bottom_right = domain.create_region('Bottom_Right', 'vertex 1', 'vertex')

field = Field.from_args('fu', np.float64, 'vector', omega,
                        approx_order=2)

u = FieldVariable('u', 'unknown', field)
v = FieldVariable('v', 'test', field, primary_var_name='u')

# define elastic properties:
# Aluminum properties in SI units
E = 7.0e10  # Young's Modulus in Pa (70 GPa for Aluminum)
nu = 0.33  # Poisson's ratio for Aluminum
D = stiffness_from_youngpoisson(2, young=E, poisson=nu)

aluminum = Material('Aluminum', D=D)

#set up loading:
# Define forces in Newtons
fy = -6e3  # 1 kn is 1e3 in negative y direction (down).
force_ratio = np.tan(np.radians(10))  # Work with an angle of 10 degrees.
fx = -fy * force_ratio  # Horizontal force in positive x direction
load = Material('Load', values={'.val' : [fx, fy]})

integral = Integral('i', order=2)
integral0 = Integral('i', order=0)

t1 = Term.new('dw_lin_elastic(Aluminum.D, v, u)',
                integral, omega, Aluminum=aluminum, v=v, u=u)
t2 = Term.new('dw_point_load(Load.val, v)',
                integral0, top, Load=load, v=v)
eq = Equation('balance', t1 + t2)
eqs = Equations([eq])

ls = ScipyDirect({})

nls_status = IndexedStruct()
nls = Newton({}, lin_solver=ls, status=nls_status)

pb = Problem('elasticity', equations=eqs)

# Apply boundary conditions to the two bottom vertices
fix_bottom_left = EssentialBC('fix_bottom_left', bottom_left, {'u.all': 0.0})  # Fix both x and y displacements
fix_bottom_right = EssentialBC('fix_bottom_right', bottom_right, {'u.all': 0.0})  # Fix both x and y displacements

# Add the boundary conditions to the problem setup
pb.set_bcs(ebcs=Conditions([fix_bottom_left, fix_bottom_right]))

pb.set_solver(nls)

# Solve the problem.
variables = pb.solve()
output(nls_status)

# Postprocess the solution.
out = variables.create_output()

ev = pb.evaluate
strain = ev('ev_cauchy_strain.2.Omega(u)', mode='el_avg')
stress = ev('ev_cauchy_stress.2.Omega(Aluminum.D, u)', mode='el_avg',
                copy_materials=False)

out['cauchy_strain'] = Struct(name='output_data', mode='cell',
                                  data=strain, dofs=None)
out['cauchy_stress'] = Struct(name='output_data', mode='vector',
                                  data=stress, dofs=None)

pb.save_state('triangle_load.vtk', out=out)

## Calculate nodal stresses.

# Setup
gdata = geometry_data['2_3']
nc = len(gdata.coors)

integral_vn = Integral('ivn', coors=gdata.coors,
                        weights=[gdata.volume / nc] * nc)

integrals = Integrals([integral_vn])

# Update problem time
pb.time_update()

# Evaluate stress at quadrature points
stress1 = pb.evaluate('ev_cauchy_stress.ivn.Omega(Aluminum.D, u)', mode='qp', 
                        integrals=integrals, copy_materials=False)

# Create stress field variable from quadrature points
sfield = Field.from_args('stress1', np.float64, (3,), omega)
svar = FieldVariable('sigma', 'parameter', sfield, 
                        primary_var_name='(set-to-None)')
svar.set_from_qp(stress1, integrals['ivn'])


sfepy:     reading mesh (cm_triangle2d.mesh)...
sfepy:       number of vertices: 279
sfepy:       number of cells:
sfepy:         2_3: 487
sfepy:     ...done in 0.00 s
sfepy:     updating variables...
sfepy:     ...done
sfepy:     setting up dof connectivities...
sfepy:     ...done in 0.00 s
sfepy:     matrix shape: (2084, 2084)
sfepy:     assembling matrix graph...
sfepy:     ...done in 0.00 s
sfepy:     matrix structural nonzeros: 45800 (1.05e+00% fill)
sfepy:     updating variables...
sfepy:     ...done
sfepy:     updating materials...
sfepy:         Load
sfepy:         Aluminum
sfepy:     ...done in 0.00 s
sfepy:     nls: iter: 0, residual: 6.092560e+03 (rel: 1.000000e+00)
sfepy:       residual:    0.00 [s]
sfepy:         matrix:    0.00 [s]
sfepy:          solve:    0.01 [s]
sfepy:     warning: linear system solution precision is lower then the value set in solver options! (err = 7.599456e-10 < 1.000000e-10)
sfepy:     nls: iter: 1, residual: 7.764315e-10 (rel: 1.274393e-13)


Warning: VTK requires 3D vectors, but 2D vectors given. Appending 0 third component to u.

sfepy:     solved in 1 steps in 0.02 seconds
sfepy:     IndexedStruct
  condition:
    1
  err:
    7.764314679064477e-10
  err0:
    6092.5596713144705
  ls_n_iter:
    -1
  n_iter:
    1
  time:
    0.013016417040489614
  time_stats:
    dict with keys: ['residual', 'matrix', 'solve']
sfepy:     equation "tmp":
sfepy:     ev_cauchy_strain.2.Omega(u)
sfepy:     updating materials...
sfepy:     ...done in 0.00 s
sfepy:     equation "tmp":
sfepy:     ev_cauchy_stress.2.Omega(Aluminum.D, u)
sfepy:     updating materials...
sfepy:         Aluminum
sfepy:     ...done in 0.00 s


Warning: VTK requires 3D vectors, but 2D vectors given. Appending 0 third component to u.

sfepy:     updating variables...
sfepy:     ...done
sfepy:     equation "tmp":
sfepy:     ev_cauchy_stress.ivn.Omega(Aluminum.D, u)
sfepy:     updating materials...
sfepy:         Aluminum
sfepy:     ...done in 0.00 s


In [153]:
# Assuming 'svar' is your FieldVariable and 'Omega' is a region in your domain:
stress_at_nodes = svar.get_state_in_region(omega, reshape=True)
print(stress_at_nodes)

[[  2808848.0765256   15493220.65235461   5698561.04019556]
 [ 12824872.99606878  10007663.58653576 -10020633.55031441]
 [  2219484.00585672  60352659.97294801 -10274822.25199117]
 ...
 [  1815688.79330241   1178540.91106925  -1480382.85350555]
 [   825085.632021      300378.07860351    158345.61582387]
 [   177779.39552437   1177637.34603807   -458675.92156547]]


In [154]:
svar.data[0]

array([ 2808848.0765256 , 15493220.65235461,  5698561.04019556, ...,
         177779.39552437,  1177637.34603807,  -458675.92156547])

In [155]:
fx, fy

(1057.9618842507898, -6000.0)

In [156]:
s_x, s_y = stress_at_nodes[0, 0:2]
s_x, s_y

(2808848.076525604, 15493220.652354605)

In [157]:
np.rad2deg(np.arctan2(s_y, s_x))

79.72415646810038

In [158]:
stress2 = pb.evaluate('ev_cauchy_stress.ivn.Omega(Aluminum.D, u)', mode='el_avg', 
                        integrals=integrals, copy_materials=False)

sfepy:     equation "tmp":
sfepy:     ev_cauchy_stress.ivn.Omega(Aluminum.D, u)
sfepy:     updating materials...
sfepy:         Aluminum
sfepy:     ...done in 0.00 s


In [159]:
stress_at_nodes.shape

(279, 3)

In [160]:
# Define a 2D fields for compressive stress that is order=0 so that it is over cells:
compressive_field1 = Field.from_args('stress-compression', np.float64, (2,), omega, approx_order=1)
compressive_field2 = Field.from_args('stress-compression', np.float64, (2,), omega, approx_order=0)

In [161]:
xy_stress1 = stress_at_nodes[:, [0, 1]]
xy_stress2 = stress[:, :, 0:2, 0]
compressive_stress_data = xy_stress1.flatten()

In [162]:
compressive_stress_data.shape

(558,)

In [163]:
print(compressive_field1)

H1NodalVolumeField:stress-compression
  approx_order:
    1
  basis_transform:
    None
  bf:
    dict with keys: []
  bubble_dofs:
    None
  bubble_remap:
    None
  cmesh:
    CMesh: n_coor: 279, dim 2, tdim: 2, n_el 487
  coors:
    (279, 2) array of float64
  domain:
    FEDomain:domain
  dtype:
    <class 'numpy.float64'>
  econn:
    (487, 3) array of int32
  econn0:
    None
  edge_dofs:
    None
  edge_remap:
    None
  efaces:
    (3, 2) array of int32
  extra_data:
    dict with keys: []
  face_dofs:
    None
  face_remap:
    None
  force_bubble:
    False
  gel:
    GeometryElement:2_3
  is_surface:
    False
  mappings:
    dict with keys: []
  mappings0:
    dict with keys: []
  n_bubble_dof:
    0
  n_components:
    2
  n_edge_dof:
    0
  n_face_dof:
    0
  n_nod:
    279
  n_vertex_dof:
    279
  name:
    stress-compression
  node_desc:
    NodeDescription
  ori:
    None
  poly_space:
    LagrangeSimplexPolySpace:2_3_H1_lagrange_1
  poly_space_base:
    lagrange
 

In [164]:
# Create a FieldVariable for the compressive stress (vector field)
from sfepy.discrete import FieldVariable
compressive_stress_var = FieldVariable('compressive_stress', 'parameter', compressive_field1, primary_var_name='(set-to-None)')

# Set data for the vector field
compressive_stress_var.set_data(compressive_stress_data)

# Save as a VTK file for visualization
# compressive_field.save_as_mesh('compressive_stress_vector.vtk')

In [165]:
out = compressive_stress_var.create_output()

In [166]:
pb.save_state('triangle_compressive_stress.vtk', out=out)

Warning: VTK requires 3D vectors, but 2D vectors given. Appending 0 third component to compressive_stress.